Get Match Data

In [1041]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report

In [1042]:
connection = sqlite3.connect('lol_database.db')
cursor = connection.cursor()
query = f"PRAGMA table_info(Match_Data)"
cursor.execute(query)
cols_info = cursor.fetchall()
cols = [column[1] for column in cols_info]

cursor.execute('SELECT * FROM Match_Data')
rows = cursor.fetchmany(1000)
connection.close()

df = pd.DataFrame(data = rows, columns=cols)
print(df.shape)

(1000, 288)


In [1043]:
bad_cols = ['threeWardsOneSweeperCount','thirdInhibitorDestroyedTime','teleportTakedowns','baronBuffGoldAdvantageOverThreshold', 'nexusLost',
            'shortestTimeToAceFromFirstTakedown','mythicItemUsed','fastestLegendary','riotIdName','highestCrowdControlScore','highestWardKills','earliestDragonTakedown',
            'highestChampionDamage','unseenRecalls','twoWardsOneSweeperCount','twentyMinionsIn3SecondsCount',
            'takedownsInAlcove','survivedThreeImmobilizesInFight','survivedSingleDigitHpCount','snowballsHit','quickCleanse','poroExplosions',
            'playedChampSelectPosition','outnumberedNexusKill','outerTurretExecutesBefore10Minutes','multiKillOneSpell','mejaisFullStackInTime','legendaryCount',
            'knockEnemyIntoTeamAndKill','killsOnRecentlyHealedByAramPack','killedChampTookFullTeamDamageSurvived','gameLength','elderDragonMultikills',
            'elderDragonKillsWithOpposingSoul','doubleAces','deathsByEnemyChamps','completeSupportQuestInTime','blastConeOppositeOpponentCount','InfernalScalePickup',
            'gameMode','unrealKills','teamId','teamEarlySurrendered','subteamPlacement','role','riotIdTagline',
            'riotIdGameName','profileIcon','playerSubteamId','placement','participantId','nexusTakedowns','nexusKills','lane','individualPosition',
            'gameEndedInSurrender','firstTowerAssist','eligibleForProgression','championTransform','championId', 'item0','item1','item2',
            'item3','item4','item5','item6', 'sightWardsBoughtInGame']
# need to decrypt Id or remove data from df
summSpells = ['summoner1Casts', 'summoner1Id','summoner2Casts','summoner2Id']

In [1044]:
keyWords = ['playerAugment','playerScore','summoner','spell']
for col in cols:
    for key in keyWords:
        if key in col:
            df.pop(col)
print(df.shape)


(1000, 247)


In [1045]:
for col in bad_cols:
    df.pop(col)
print(df.shape)

(1000, 179)


PREPARING  DATA

In [1046]:
#mutate ping data
df['pingsTotal'] = (df['allInPings']+ df['assistMePings']+ df['basicPings']+
                       df['commandPings']+df['dangerPings']+df['enemyMissingPings']+
                       df['enemyVisionPings']+df['getBackPings']+df['holdPings']+
                       df['needVisionPings']+df['onMyWayPings']+df['pushPings']+
                       df['visionClearedPings']+df['baitPings'])

C:\Users\niska\AppData\Local\Temp\ipykernel_41436\1423892336.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['pingsTotal'] = (df['allInPings']+ df['assistMePings']+ df['basicPings']+


In [1047]:
for col in df.columns:
    if 'Ping' in col:
        df.pop(col)

In [1048]:
print(df['hadAfkTeammate'][0])
df = df.replace(np.nan, 0)
print(df['hadAfkTeammate'][0])
df = df.drop_duplicates()

nan
0.0


CREATING TEAM SUMMARY DATA FOR MACHINE LEARNING

In [1049]:
df_win = df[df['win'] == 1]
df_loss = df[df['win'] == 0]

df_topW = df_win[df_win['teamPosition'] == 'TOP']
df_topL = df_loss[df_loss['teamPosition'] == 'TOP']

team_drops = ['champLevel','championName','damageDealtToBuildings','damageDealtToTurrets','largestCriticalStrike','largestKillingSpree',
              'largestMultiKill','longestTimeSpentLiving','puuid','damageTakenOnTeamPercentage',
              'kda','killParticipation','takedowns','teamDamagePercentage','visionScoreAdvantageLaneOpponent']
for drop in team_drops:
    if drop in df_win.columns:
        df_win.pop(drop)
        df_loss.pop(drop)

In [1050]:
df_winAgg = df_topW.groupby('matchId').agg({
    'gameEndedInEarlySurrender' : 'median',
    'win' : 'median',
    'kills' : 'sum',
    'deaths' : 'sum',
    'assists' : 'sum'
})

'''
'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'damageDealtToObjectives' : 'sum', 'deaths' : 'sum', 'dragonKills' : 'sum',
    'inhibitorKills' : 'sum', 'kills' : 'sum', 'killsNearEnemyTurret' : 'sum', 'gameEndedInEarlySurrender' : 'median', 'win' : 'median'
'''

'''
df_winAgg = df_win.groupby('matchId').agg({
    'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'champExperience' : 'sum', 'consumablesPurchased' : 'sum',  
    'damageDealtToObjectives' : 'sum', 'damageSelfMitigated' : 'sum', 'deaths' : 'sum', 'detectorWardsPlaced' : 'sum', 'doubleKills' : 'sum',
    'dragonKills' : 'sum', 'firstBloodAssist' : 'sum', 'firstBloodKill' : 'sum', 'firstTowerKill' : 'sum', 'goldEarned' : 'sum',
    'goldSpent' : 'sum', 'inhibitorKills' : 'sum', 'inhibitorTakedowns' : 'sum', 'inhibitorsLost' : 'mean', 'itemsPurchased' : 'sum',
    'killingSprees' : 'sum', 'kills' : 'sum', 'magicDamageDealt' : 'sum', 'magicDamageDealtToChampions' : 'sum', 'magicDamageTaken' : 'sum',
    'neutralMinionsKilled' : 'sum', 'objectivesStolenAssists' : 'sum', 'pentaKills' : 'sum', 'physicalDamageDealt' : 'sum', 
    'physicalDamageDealtToChampions' : 'sum', 'physicalDamageTaken' : 'sum', 'quadraKills' : 'sum',
    'timeCCingOthers' : 'sum', 'totalAllyJungleMinionsKilled' : 'sum', 'totalDamageDealt' : 'sum', 'totalDamageDealtToChampions' : 'sum', 
    'totalDamageShieldedOnTeammates' : 'sum', 'totalDamageTaken' : 'sum', 'totalEnemyJungleMinionsKilled' : 'sum', 'totalHeal' : 'sum', 
    'totalHealsOnTeammates' : 'sum', 'totalMinionsKilled' : 'sum', 'totalTimeCCDealt' : 'sum', 'totalTimeSpentDead' : 'sum', 
    'totalUnitsHealed' : 'sum', 'tripleKills' : 'sum', 'trueDamageDealt' : 'sum', 'trueDamageDealtToChampions' : 'sum', 'trueDamageTaken' : 'sum',
    'turretKills' : 'sum', 'turretTakedowns' : 'sum', 'turretsLost' : 'median', 'visionScore' : 'sum', 'visionWardsBoughtInGame' : 'sum', 
    'wardsKilled' : 'sum', 'wardsPlaced' : 'sum', 'win' : 'median', '12AssistStreakCount' : 'sum', 'abilityUses' : 'sum', 'acesBefore15Minutes' : 'median', 
    'alliedJungleMonsterKills': 'sum', 'baronTakedowns' : 'sum', 'bountyGold' : 'sum', 'buffsStolen' : 'sum', 'controlWardTimeCoverageInRiverOrEnemyHalf' : 'sum',
    'controlWardsPlaced' : 'sum', 'damagePerMinute' : 'sum', 'dancedWithRiftHerald' : 'sum', 'dodgeSkillShotsSmallWindow' : 'sum', 'dragonTakedowns' : 'sum', 
    'earlyLaningPhaseGoldExpAdvantage' : 'sum', 'effectiveHealAndShielding' : 'sum', 'enemyChampionImmobilizations' : 'sum', 
    'enemyJungleMonsterKills' : 'sum', 'epicMonsterKillsNearEnemyJungler' : 'sum', 'epicMonsterKillsWithin30SecondsOfSpawn' : 'sum',
    'epicMonsterSteals' : 'sum', 'epicMonsterStolenWithoutSmite' : 'sum', 'firstTurretKilled' : 'median', 'flawlessAces' : 'median', 
    'fullTeamTakedown' : 'median', 'goldPerMinute' : 'sum', 'hadOpenNexus' : 'median', 'immobilizeAndKillWithAlly' : 'sum', 'initialBuffCount' : 'sum', 
    'initialCrabCount' : 'sum', 'jungleCsBefore10Minutes' : 'sum', 'junglerTakedownsNearDamagedEpicMonster' : 'sum', 'kTurretsDestroyedBeforePlatesFall' : 'sum',
    'killAfterHiddenWithAlly' : 'sum', 'killsNearEnemyTurret' : 'sum', 'getTakedownsInAllLanesEarlyJungleAsLaner': 'sum', 
    'killsOnOtherLanesEarlyJungleAsLaner' : 'sum', 'killsUnderOwnTurret' : 'sum', 'killsWithHelpFromEpicMonster' : 'sum', 
    'landSkillShotsEarlyGame' : 'sum', 'laneMinionsFirst10Minutes' : 'sum', 'laningPhaseGoldExpAdvantage' : 'sum', 'lostAnInhibitor' : 'median',
    'maxCsAdvantageOnLaneOpponent' : 'sum', 'maxKillDeficit' : 'sum', 'maxLevelLeadLaneOpponent' : 'sum', 
    'moreEnemyJungleThanOpponent' : 'sum', 'multiTurretRiftHeraldCount' : 'median', 'multikills' : 'sum', 'multikillsAfterAggressiveFlash' : 'sum',
    'perfectDragonSoulsTaken' : 'median', 'perfectGame' : 'median', 'pickKillWithAlly' : 'sum', 'quickFirstTurret' : 'sum', 'quickSoloKills' : 'sum',
    'riftHeraldTakedowns' : 'sum', 'saveAllyFromDeath' : 'sum', 'scuttleCrabKills' : 'sum', 'skillshotsDodged' : 'sum', 'skillshotsHit' : 'sum',
    'soloKills' : 'sum', 'soloTurretsLategame' : 'sum', 'stealthWardsPlaced' : 'sum', 'takedownOnFirstTurret' : 'sum', 
    'takedownsAfterGainingLevelAdvantage' : 'sum', 'takedownsBeforeJungleMinionSpawn' : 'sum', 'takedownsFirstXMinutes' : 'sum', 
    'takedownsInEnemyFountain' : 'sum', 'teamBaronKills' : 'median', 'teamElderDragonKills' : 'median', 'teamRiftHeraldKills' : 'median',
    'tookLargeDamageSurvived' : 'sum', 'turretPlatesTaken' : 'sum', 'turretsTakenWithRiftHerald' : 'median', 'visionScorePerMinute' : 'sum',
    'wardTakedowns' : 'sum', 'wardTakedownsBefore20M' : 'sum', 'wardsGuarded' : 'sum', 'junglerKillsEarlyJungle' : 'sum', 
    'killsOnLanersEarlyJungleAsJungler' : 'sum', 'earliestBaron' : 'median', 'firstTurretKilledTime' : 'median', 'fasterSupportQuestCompletion' : 'sum',
    'fistBumpParticipation' : 'sum', 'voidMonsterKill' : 'sum', 'hadAfkTeammate' : 'median', 'earliestElderDragon' : 'median', 'gameEndedInEarlySurrender' : 'median'
    })
'''

"\ndf_winAgg = df_win.groupby('matchId').agg({\n    'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'champExperience' : 'sum', 'consumablesPurchased' : 'sum',  \n    'damageDealtToObjectives' : 'sum', 'damageSelfMitigated' : 'sum', 'deaths' : 'sum', 'detectorWardsPlaced' : 'sum', 'doubleKills' : 'sum',\n    'dragonKills' : 'sum', 'firstBloodAssist' : 'sum', 'firstBloodKill' : 'sum', 'firstTowerKill' : 'sum', 'goldEarned' : 'sum',\n    'goldSpent' : 'sum', 'inhibitorKills' : 'sum', 'inhibitorTakedowns' : 'sum', 'inhibitorsLost' : 'mean', 'itemsPurchased' : 'sum',\n    'killingSprees' : 'sum', 'kills' : 'sum', 'magicDamageDealt' : 'sum', 'magicDamageDealtToChampions' : 'sum', 'magicDamageTaken' : 'sum',\n    'neutralMinionsKilled' : 'sum', 'objectivesStolenAssists' : 'sum', 'pentaKills' : 'sum', 'physicalDamageDealt' : 'sum', \n    'physicalDamageDealtToChampions' : 'sum', 'physicalDamageTaken' : 'sum', 'quadraKills' : 'sum',\n    'timeCCingOthers' : 'sum', 'totalAllyJung

In [1051]:
df_lossAgg = df_loss.groupby('matchId').agg({
    'gameEndedInEarlySurrender' : 'median',
    'win' : 'median',
    'kills': 'sum',
    'deaths' : 'sum',
    'assists' : 'sum'
})

'''
'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'damageDealtToObjectives' : 'sum', 'deaths' : 'sum', 'dragonKills' : 'sum',
    'inhibitorKills' : 'sum', 'kills' : 'sum', 'killsNearEnemyTurret' : 'sum', 'gameEndedInEarlySurrender' : 'median', 'win' : 'median'
'''

'''
df_lossAgg = df_loss.groupby('matchId').agg({
    'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'champExperience' : 'sum', 'consumablesPurchased' : 'sum',
    'damageDealtToObjectives' : 'sum', 'damageSelfMitigated' : 'sum', 'deaths' : 'sum', 'detectorWardsPlaced' : 'sum', 'doubleKills' : 'sum',
    'dragonKills' : 'sum', 'firstBloodAssist' : 'sum', 'firstBloodKill' : 'sum', 'firstTowerKill' : 'sum', 'goldEarned' : 'sum',
    'goldSpent' : 'sum', 'inhibitorKills' : 'sum', 'inhibitorTakedowns' : 'sum', 'inhibitorsLost' : 'mean', 'itemsPurchased' : 'sum',
    'killingSprees' : 'sum', 'kills' : 'sum', 'magicDamageDealt' : 'sum', 'magicDamageDealtToChampions' : 'sum', 'magicDamageTaken' : 'sum',
    'neutralMinionsKilled' : 'sum', 'objectivesStolenAssists' : 'sum', 'pentaKills' : 'sum', 'physicalDamageDealt' : 'sum', 
    'physicalDamageDealtToChampions' : 'sum', 'physicalDamageTaken' : 'sum', 'quadraKills' : 'sum',
    'timeCCingOthers' : 'sum', 'totalAllyJungleMinionsKilled' : 'sum', 'totalDamageDealt' : 'sum', 'totalDamageDealtToChampions' : 'sum', 
    'totalDamageShieldedOnTeammates' : 'sum', 'totalDamageTaken' : 'sum', 'totalEnemyJungleMinionsKilled' : 'sum', 'totalHeal' : 'sum', 
    'totalHealsOnTeammates' : 'sum', 'totalMinionsKilled' : 'sum', 'totalTimeCCDealt' : 'sum', 'totalTimeSpentDead' : 'sum', 
    'totalUnitsHealed' : 'sum', 'tripleKills' : 'sum', 'trueDamageDealt' : 'sum', 'trueDamageDealtToChampions' : 'sum', 'trueDamageTaken' : 'sum',
    'turretKills' : 'sum', 'turretTakedowns' : 'sum', 'turretsLost' : 'median', 'visionScore' : 'sum', 'visionWardsBoughtInGame' : 'sum', 
    'wardsKilled' : 'sum', 'wardsPlaced' : 'sum', 'win' : 'median', '12AssistStreakCount' : 'sum', 'abilityUses' : 'sum', 'acesBefore15Minutes' : 'median', 
    'alliedJungleMonsterKills': 'sum', 'baronTakedowns' : 'sum', 'bountyGold' : 'sum', 'buffsStolen' : 'sum', 'controlWardTimeCoverageInRiverOrEnemyHalf' : 'sum',
    'controlWardsPlaced' : 'sum', 'damagePerMinute' : 'sum', 'dancedWithRiftHerald' : 'sum', 'dodgeSkillShotsSmallWindow' : 'sum', 'dragonTakedowns' : 'sum', 
    'earlyLaningPhaseGoldExpAdvantage' : 'sum', 'effectiveHealAndShielding' : 'sum', 'enemyChampionImmobilizations' : 'sum', 
    'enemyJungleMonsterKills' : 'sum', 'epicMonsterKillsNearEnemyJungler' : 'sum', 'epicMonsterKillsWithin30SecondsOfSpawn' : 'sum',
    'epicMonsterSteals' : 'sum', 'epicMonsterStolenWithoutSmite' : 'sum', 'firstTurretKilled' : 'median', 'flawlessAces' : 'median', 
    'fullTeamTakedown' : 'median', 'goldPerMinute' : 'sum', 'hadOpenNexus' : 'median', 'immobilizeAndKillWithAlly' : 'sum', 'initialBuffCount' : 'sum', 
    'initialCrabCount' : 'sum', 'jungleCsBefore10Minutes' : 'sum', 'junglerTakedownsNearDamagedEpicMonster' : 'sum', 'kTurretsDestroyedBeforePlatesFall' : 'sum',
    'killAfterHiddenWithAlly' : 'sum', 'killsNearEnemyTurret' : 'sum', 'getTakedownsInAllLanesEarlyJungleAsLaner': 'sum', 
    'killsOnOtherLanesEarlyJungleAsLaner' : 'sum', 'killsUnderOwnTurret' : 'sum', 'killsWithHelpFromEpicMonster' : 'sum', 
    'landSkillShotsEarlyGame' : 'sum', 'laneMinionsFirst10Minutes' : 'sum', 'laningPhaseGoldExpAdvantage' : 'sum', 'lostAnInhibitor' : 'median',
    'maxCsAdvantageOnLaneOpponent' : 'sum', 'maxKillDeficit' : 'sum', 'maxLevelLeadLaneOpponent' : 'sum', 
    'moreEnemyJungleThanOpponent' : 'sum', 'multiTurretRiftHeraldCount' : 'median', 'multikills' : 'sum', 'multikillsAfterAggressiveFlash' : 'sum',
    'perfectDragonSoulsTaken' : 'median', 'perfectGame' : 'median', 'pickKillWithAlly' : 'sum', 'quickFirstTurret' : 'sum', 'quickSoloKills' : 'sum',
    'riftHeraldTakedowns' : 'sum', 'saveAllyFromDeath' : 'sum', 'scuttleCrabKills' : 'sum', 'skillshotsDodged' : 'sum', 'skillshotsHit' : 'sum',
    'soloKills' : 'sum', 'soloTurretsLategame' : 'sum', 'stealthWardsPlaced' : 'sum', 'takedownOnFirstTurret' : 'sum', 
    'takedownsAfterGainingLevelAdvantage' : 'sum', 'takedownsBeforeJungleMinionSpawn' : 'sum', 'takedownsFirstXMinutes' : 'sum', 
    'takedownsInEnemyFountain' : 'sum', 'teamBaronKills' : 'median', 'teamElderDragonKills' : 'median', 'teamRiftHeraldKills' : 'median',
    'tookLargeDamageSurvived' : 'sum', 'turretPlatesTaken' : 'sum', 'turretsTakenWithRiftHerald' : 'median', 'visionScorePerMinute' : 'sum',
    'wardTakedowns' : 'sum', 'wardTakedownsBefore20M' : 'sum', 'wardsGuarded' : 'sum', 'junglerKillsEarlyJungle' : 'sum', 
    'killsOnLanersEarlyJungleAsJungler' : 'sum', 'earliestBaron' : 'median', 'firstTurretKilledTime' : 'median', 'fasterSupportQuestCompletion' : 'sum',
    'fistBumpParticipation' : 'sum', 'voidMonsterKill' : 'sum', 'hadAfkTeammate' : 'median', 'earliestElderDragon' : 'median', 'gameEndedInEarlySurrender' : 'median' 
    })
'''

"\ndf_lossAgg = df_loss.groupby('matchId').agg({\n    'assists' : 'sum', 'baronKills' : 'sum', 'bountyLevel' : 'sum', 'champExperience' : 'sum', 'consumablesPurchased' : 'sum',\n    'damageDealtToObjectives' : 'sum', 'damageSelfMitigated' : 'sum', 'deaths' : 'sum', 'detectorWardsPlaced' : 'sum', 'doubleKills' : 'sum',\n    'dragonKills' : 'sum', 'firstBloodAssist' : 'sum', 'firstBloodKill' : 'sum', 'firstTowerKill' : 'sum', 'goldEarned' : 'sum',\n    'goldSpent' : 'sum', 'inhibitorKills' : 'sum', 'inhibitorTakedowns' : 'sum', 'inhibitorsLost' : 'mean', 'itemsPurchased' : 'sum',\n    'killingSprees' : 'sum', 'kills' : 'sum', 'magicDamageDealt' : 'sum', 'magicDamageDealtToChampions' : 'sum', 'magicDamageTaken' : 'sum',\n    'neutralMinionsKilled' : 'sum', 'objectivesStolenAssists' : 'sum', 'pentaKills' : 'sum', 'physicalDamageDealt' : 'sum', \n    'physicalDamageDealtToChampions' : 'sum', 'physicalDamageTaken' : 'sum', 'quadraKills' : 'sum',\n    'timeCCingOthers' : 'sum', 'totalAllyJung

visualize data

In [1052]:
'''
for label in df_winAgg.columns:
    plt.hist(df_winAgg[label], color = 'blue', label = 'win', alpha =0.7, density = True)
    plt.hist(df_lossAgg[label], color = 'red', label = 'loss', alpha =0.7, density = True)
    plt.title(label)
    plt.ylabel('probability')
    plt.xlabel(label)
    plt.legend()
    plt.show()
'''

"\nfor label in df_winAgg.columns:\n    plt.hist(df_winAgg[label], color = 'blue', label = 'win', alpha =0.7, density = True)\n    plt.hist(df_lossAgg[label], color = 'red', label = 'loss', alpha =0.7, density = True)\n    plt.title(label)\n    plt.ylabel('probability')\n    plt.xlabel(label)\n    plt.legend()\n    plt.show()\n"

scale data and create train, valid, and test datasets

In [1053]:
#creating W/L dataframe
dfWL = pd.concat([df_winAgg, df_lossAgg])
print(dfWL.shape)

#removing remakes
dfWL = dfWL[dfWL['gameEndedInEarlySurrender'] == 0]
dfWL.pop('gameEndedInEarlySurrender')
print(dfWL.shape)


(200, 5)
(190, 4)


In [1054]:
train, valid, test = np.split(dfWL.sample(frac=1), [int(0.6*len(dfWL)), int(0.8*len(dfWL))])

c:\dev\lol_data_project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [1055]:
print(len(train[train['win'] == 1]))
print(len(train[train['win'] == 0]))

58
56


In [1056]:
def scale_data(data, oversample=False): 

    x = data.drop(columns = 'win').values
    y = data['win'].values

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    if oversample:
        ros = RandomOverSampler()
        x,y = ros.fit_resample(x,y)
    data = np.hstack((x, np.reshape(y, (-1,1))))

    return data, x, y


In [1057]:
train, x_train, y_train = scale_data(train, oversample=True)
valid, x_valid, y_valid = scale_data(valid)
test, x_test, y_test = scale_data(test)

NAIVE BAYES

In [1058]:
from sklearn.naive_bayes import GaussianNB

In [1059]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X = x_train, y= y_train)

In [1060]:
y_pred = nb_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.86      0.92        21
         1.0       0.85      1.00      0.92        17

    accuracy                           0.92        38
   macro avg       0.93      0.93      0.92        38
weighted avg       0.93      0.92      0.92        38



CREATING POSITIONAL DATA FOR MACHINE LEARNING

In [1061]:
df = df[df['gameEndedInEarlySurrender'] == 0]

df_win = df[df['win'] == 1]
df_loss = df[df['win'] == 0]

df_topW = df_win[df_win['teamPosition'] == 'TOP']
df_topL = df_loss[df_loss['teamPosition'] == 'TOP']

df_midW = df_win[df_win['teamPosition'] == 'MIDDLE']
df_midL = df_loss[df_loss['teamPosition'] == 'MIDDLE']

df_jgW = df_win[df_win['teamPosition'] == 'JUNGLE']
df_jgL = df_loss[df_loss['teamPosition'] == 'JUNGLE']

df_botW = df_win[df_win['teamPosition'] == 'BOTTOM']
df_botL = df_loss[df_loss['teamPosition'] == 'BOTTOM']

df_supW = df_win[df_win['teamPosition'] == 'UTILITY']
df_supL = df_loss[df_loss['teamPosition'] == 'UTILITY']

TOP

In [1062]:
df_topW = df_topW[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold',
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

df_topL = df_topL[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold', 
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

MIDDLE

In [1063]:
df_midW = df_midW[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold',
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

df_midL = df_midL[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold', 
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

JUNGLE

In [1064]:
df_jgW = df_jgW[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold',
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

df_jgL = df_jgL[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold', 
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

BOTTOM

In [1065]:
df_botW = df_botW[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold',
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

df_botL = df_botL[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold', 
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

SUPPORT

In [1066]:
df_supW = df_supW[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold',
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

df_supL = df_supL[['kills', 'deaths', 'assists', 'gameEndedInEarlySurrender','win',
                   'champExperience', 'bountyLevel', 'bountyGold', 
                   'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
                   'damageSelfMitigated', 'goldEarned', 'largestKillingSpree',
                   'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled',
                   'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken',
                   'timeCCingOthers', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken',
                   'totalDamageShieldedOnTeammates', 'totalHeal', 'totalMinionsKilled', 'totalTimeCCDealt',
                   'totalTimeSpentDead', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
                   'turretKills', 'visionScore', 'wardsKilled', 'wardsPlaced', 'damagePerMinute',
                   'damageTakenOnTeamPercentage', 'earlyLaningPhaseGoldExpAdvantage', 
                   'kda', 'killParticipation', 'killsNearEnemyTurret', 'laneMinionsFirst10Minutes',
                   'laningPhaseGoldExpAdvantage', 'maxCsAdvantageOnLaneOpponent', 'maxKillDeficit',
                   'maxLevelLeadLaneOpponent', 'soloKills', 'soloTurretsLategame', 'teamDamagePercentage',
                   'turretPlatesTaken']]

In [1067]:
'''
for label in df_topW.columns:
    plt.hist(df_topW[label], color = 'blue', label = 'win', alpha =0.7, density = True)
    plt.hist(df_topL[label], color = 'red', label = 'loss', alpha =0.7, density = True)
    plt.title(label)
    plt.ylabel('probability')
    plt.xlabel(label)
    plt.legend()
    plt.show()
'''

"\nfor label in df_topW.columns:\n    plt.hist(df_topW[label], color = 'blue', label = 'win', alpha =0.7, density = True)\n    plt.hist(df_topL[label], color = 'red', label = 'loss', alpha =0.7, density = True)\n    plt.title(label)\n    plt.ylabel('probability')\n    plt.xlabel(label)\n    plt.legend()\n    plt.show()\n"

In [1068]:
#creating W/L dataframe
dfTopWL = pd.concat([df_topW, df_topL])

dfMidWL = pd.concat([df_midW, df_midL])

dfJgWL = pd.concat([df_jgW, df_jgL])

dfBotWL = pd.concat([df_botW, df_botL])

dfSupWL = pd.concat([df_supW, df_supL])


In [1069]:
#top
Ttrain, Tvalid, Ttest = np.split(dfTopWL.sample(frac=1), [int(0.6*len(dfTopWL)), int(0.8*len(dfTopWL))])
trainTop, x_trainTop, y_trainTop = scale_data(Ttrain, oversample=True)
validTop, x_validTop, y_validTop = scale_data(Tvalid)
testTop, x_testTop, y_testTop = scale_data(Ttest)

#mid
Mtrain, Mvalid, Mtest = np.split(dfMidWL.sample(frac=1), [int(0.6*len(dfMidWL)), int(0.8*len(dfMidWL))])
trainMid, x_trainMid, y_trainMid = scale_data(Mtrain, oversample=True)
validMid, x_validMid, y_validMid = scale_data(Mvalid)
testMid, x_testMid, y_testMid = scale_data(Mtest)

#jg
Jtrain, Jvalid, Jtest = np.split(dfJgWL.sample(frac=1), [int(0.6*len(dfJgWL)), int(0.8*len(dfJgWL))])
trainJg, x_trainJg, y_trainJg = scale_data(Jtrain, oversample=True)
validJg, x_validJg, y_validJg = scale_data(Jvalid)
testJg, x_testJg, y_testJg = scale_data(Jtest)

#bot
Btrain, Bvalid, Btest = np.split(dfBotWL.sample(frac=1), [int(0.6*len(dfBotWL)), int(0.8*len(dfBotWL))])
trainBot, x_trainBot, y_trainBot = scale_data(Btrain, oversample=True)
validBot, x_validBot, y_validBot = scale_data(Bvalid)
testBot, x_testBot, y_testBot = scale_data(Btest)

#support
S_train, S_valid, S_test = np.split(dfSupWL.sample(frac=1), [int(0.6*len(dfSupWL)), int(0.8*len(dfSupWL))])
trainSup, x_trainSup, y_trainSup = scale_data(S_train, oversample=True)
validSup, x_validSup, y_validSup = scale_data(S_valid)
testSup, x_testSup, y_testSup = scale_data(S_test)


c:\dev\lol_data_project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\dev\lol_data_project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\dev\lol_data_project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\dev\lol_data_project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\dev\lol_data_project\venv\Lib\site-pa

In [1070]:
Top_model = GaussianNB()
Top_model = Top_model.fit(X = x_trainTop, y= y_trainTop)
y_predTop = Top_model.predict(x_testTop)
print('TOP REPORT')
print(classification_report(y_testTop, y_predTop))

Mid_model = GaussianNB()
Mid_model = Mid_model.fit(X = x_trainMid, y= y_trainMid)
y_predMid = Mid_model.predict(x_testMid)
print('MID REPORT')
print(classification_report(y_testMid, y_predMid))

Jg_model = GaussianNB()
Jg_model = Jg_model.fit(X = x_trainJg, y= y_trainJg)
y_predJg = Jg_model.predict(x_testJg)
print('Jg REPORT')
print(classification_report(y_testJg, y_predJg))

Bot_model = GaussianNB()
Bot_model = Bot_model.fit(X = x_trainBot, y= y_trainBot)
y_predBot = Bot_model.predict(x_testBot)
print('Bot REPORT')
print(classification_report(y_testBot, y_predBot))

Sup_model = GaussianNB()
Sup_model = Sup_model.fit(X = x_trainSup, y= y_trainSup)
y_predSup = Sup_model.predict(x_testSup)
print('Sup REPORT')
print(classification_report(y_testSup, y_predSup))


TOP REPORT
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.45      1.00      0.62        17

    accuracy                           0.45        38
   macro avg       0.22      0.50      0.31        38
weighted avg       0.20      0.45      0.28        38

MID REPORT
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.39      1.00      0.57        15

    accuracy                           0.39        38
   macro avg       0.20      0.50      0.28        38
weighted avg       0.16      0.39      0.22        38

Jg REPORT
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.55      1.00      0.71        21

    accuracy                           0.55        38
   macro avg       0.28      0.50      0.36        38
weighted avg       0.31      0.55      0.3

c:\dev\lol_data_project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\dev\lol_data_project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\dev\lol_data_project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\dev\lol_data_project\venv